In [2]:
# Import Data 
import pandas as pd
from pandas.io.json import json_normalize
import warnings
import json
import os
warnings.filterwarnings("ignore")


## IMPORTANTE
1. Código basado en el [link](https://www.kaggle.com/code/prathamsharma123/clean-raw-json-tweets-data) que dieron
2. No lo pude probar porque me da error de que el output excede el tamaño máximo

In [ ]:
# Read JSON file containing tweets data and removce tweets not in English

raw_tweets = pd.read_json(r'../input/farmers-protest-tweets-dataset-raw-json/farmers-protest-tweets-2021-2-4.json', lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']

users = json_normalize(raw_tweets['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)

users = pd.DataFrame(users)
users.drop_duplicates(subset=['userId'], inplace=True)

# Transform 'raw_tweets' DataFrame

# Add column for 'userId'
user_id = []
for user in raw_tweets['user']:
    uid = user['id']
    user_id.append(uid)
raw_tweets['userId'] = user_id

# Remove less important columns
cols = ['url', 'date', 'renderedContent', 'id', 'userId', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'source', 'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers']
tweets = raw_tweets[cols]
tweets.rename(columns={'id':'tweetId', 'url':'tweetUrl'}, inplace=True)

In [ ]:
def top_10_retweet(df):
    top = df['retweetCount'].nlargest(n=10)
    print(top[['tweetId', 'retweetCount']])

def top_10_user_tweets(df):
    tweets_per_user = df.groupby(['userId'])['userId'].count().reset_index(name="count")
    top = tweets_per_user['count'].nlargest(n=10)
    print(top[['userId', 'count']])

def top_10_days_tweets(df):
    tweets_per_day = df.groupby(['date'])['date'].count().reset_index(name="count")
    top = tweets_per_day['count'].nlargest(n=10)
    print(top[['date', 'count']])

In [ ]:
if __name__ == "__main__":
    print("Most Retweeted")
    top_10_retweet(tweets)

    print("Users that tweet the most")
    top_10_user_tweets(tweets)

    print("Days with most tweets")
    top_10_days_tweets(tweets)